In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np

import mysql.connector
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.insert(1, '../')

import os

In [6]:
def csv_load(ruta_carpeta):
   
    dataframes = []

    for archivo in os.listdir(ruta_carpeta):
        
        if archivo.endswith('.csv'):
            
            path = os.path.join(ruta_carpeta, archivo)

            df = pd.read_csv(path)
            
            dataframes.append((archivo.replace('.csv', ''), df))

    return dataframes

lista_dataframes = csv_load('../data')

name = []
dataframes = []
for nombre, df in lista_dataframes:
    name.append(nombre)
    dataframes.append(df)


In [7]:
dataframes

[    id                        Region   Año  Hombre  Mujer  Total
 0    1                   España (ES)  2019   77.98  81.77  79.89
 1    2                Andalucía (AN)  2019   77.81  81.11  79.48
 2    3                   Aragón (AR)  2019   79.26  82.86  81.28
 3    4  Asturias, Principado de (AS)  2019   76.94  80.58  78.81
 4    5           Balears, Illes (IB)  2019   79.92  81.06  80.46
 ..  ..                           ...   ...     ...    ...    ...
 95  16     Navarra, C. Foral de (NC)  2021   78.36  83.33  80.86
 96  17               País Vasco (PV)  2021   79.18  82.40  80.78
 97  18                 La Rioja (RI)  2021   78.00  80.87  79.60
 98  19                    Ceuta (CE)  2021   70.28  67.11  68.60
 99  20                  Melilla (ML)  2021   70.28  67.11  68.60
 
 [100 rows x 6 columns],
     id                        Region   Año  Hombre  Mujer  Total
 0    1                   España (ES)  2019   17.83  20.23  19.09
 1    2                Andalucía (AN)  2019   17.

In [ ]:
key = open('password.txt','r').read()

conexion = mysql.connector.connect(host='localhost',user='root',passwd= key ,database='VitalStatsES')
cursor = conexion.cursor()

In [ ]:
def add_table(name):
    
    try:
        try:
            df = pd.read_csv(f'../data/{name}')  

        except:
            print(f"Error: couldn't find {name} in ../data/")
            return (1)

        if f'{name}' not in df.columns:
            print(f"Error: {name}_id not in dataframe")
            return (2)

        # Downcasting so that we have smallint and floats instead of bigint and doubles

        for c in df.select_dtypes(include='float'):
            df[c] = pd.to_numeric(df[c], downcast='float')

        for c in df.select_dtypes('integer'):
            df[c] = pd.to_numeric(df[c], downcast='integer')
    
        tables_name = [csv.replace('.csv', '').replace('_', ' ').replace('Anos', 'Años') for csv in files]

        df.to_sql(name=f'{tables_name}',
                con=CURSOR,
                if_exists='append',
                index=False
               )

        with CURSOR.connect() as con:
            con.execute(f'ALTER TABLE `{name}` ADD PRIMARY KEY (`id`);')

        print("Table created succesfully")
        print('-'*15)
        return (0)
    
    except:
        print ('Unexpected error. Use help(add_table) for usage.')
        return (3)
